In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-07"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
2335,2024-06-07,Irã Superliga,09:30,Tabiat,Shahrdari Gorgan,2.29,1.58,170.5,1.85,1.85,0.0,0.00,0.00,IoULzs7s,0.436681,0.632911,0.540541,0.540541,0.069593,147.640,38.413019,0.260180,2.4,1.341641,0.559017,127.80,1.830,0.360694,0.197100,27.0,106.816,10.253508,0.095992,2.4,1.341641,0.559017,113.92,1.210,0.084558,0.069882,32.0,90,89,1.42,1.28,122.404,104.000,0.259455,0.000000,NaN,0.58,0.116,11.120690,0.699563,0.7,0.000437,-0.10,-0.020,-29.000000,0.000000,0.0,0.000000
2336,2024-06-07,Indonésia Ibl,10:00,Dewa United,Kesatria Bengawan Solo,1.25,3.69,177.5,1.85,1.85,-8.5,1.83,6.50,O6g43gdt,0.800000,0.271003,0.540541,0.540541,0.071003,120.420,22.528665,0.187084,2.4,1.341641,0.559017,135.45,1.162,0.217876,0.187501,73.0,136.354,52.800083,0.387228,2.4,1.341641,0.559017,224.25,1.678,0.893516,0.532488,11.0,129,69,1.05,3.25,118.192,132.400,0.698518,0.000000,0.792842,-0.82,-0.164,-1.524390,0.000000,0.0,0.000000,-0.07,-0.014,-192.142857,0.000000,0.0,0.000000
2337,2024-06-07,Grécia Liga De Basquete,15:30,Olympiakos,Panathinaikos,1.60,2.36,152.5,1.88,1.90,0.0,0.00,0.00,O4ETGitA,0.625000,0.423729,0.531915,0.526316,0.048729,103.364,10.328063,0.099919,3.0,0.000000,0.000000,106.68,1.128,0.092033,0.081589,107.0,98.952,11.610425,0.117334,3.0,0.000000,0.000000,95.20,1.146,0.092087,0.080355,66.0,84,80,1.27,1.19,92.180,99.186,0.271415,0.007483,NaN,0.35,0.070,8.571429,0.956326,1.0,0.043674,0.81,0.162,8.395062,0.891067,1.0,0.108933
2338,2024-06-07,Eua Wnba,23:00,Las Vegas Aces F,Seattle Storm F,1.25,4.04,167.5,1.89,1.90,0.0,0.00,0.00,WfLDj1Nh,0.800000,0.247525,0.529101,0.526316,0.047525,100.272,8.924190,0.089000,1.8,1.643168,0.912871,97.68,1.178,0.174270,0.147937,26.0,162.210,71.881237,0.443137,2.4,1.341641,0.559017,101.60,2.036,1.373401,0.674558,56.0,74,80,1.32,1.27,0.000,0.000,0.745871,0.003731,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
2339,2024-06-07,Eua Wnba,23:00,Phoenix Mercury F,Minnesota Lynx F,3.19,1.36,162.5,1.83,1.95,0.0,0.00,0.00,jcyFluh5,0.313480,0.735294,0.546448,0.512821,0.048774,271.900,184.995974,0.680382,0.6,1.341641,2.236068,237.46,3.602,2.438354,0.676944,-41.0,191.952,67.131174,0.349729,2.4,1.341641,0.559017,122.67,2.316,0.885991,0.382552,51.0,62,87,3.83,1.41,0.000,0.000,0.568794,0.044896,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
2340,2024-06-07,Eua Wnba,23:00,Los Angeles Sparks F,Dallas Wings F,2.15,1.71,160.5,1.85,1.93,1.5,1.97,1.55,C6xBkLxa,0.465116,0.584795,0.540541,0.518135,0.049912,161.208,27.661183,0.171587,0.6,1.341641,2.236068,186.32,2.202,0.400712,0.181976,-33.0,205.282,40.237870,0.196013,1.2,1.643168,1.369306,230.28,2.526,0.714164,0.282725,-2.0,68,76,2.74,3.03,0.000,205.282,0.161206,0.029930,0.168741,0.00,0.000,inf,0.000000,0.0,0.000000,-1.32,-0.264,-2.689394,0.000000,0.0,0.000000
2341,2024-06-07,Brasil Lbf Feminina,19:30,Sampaio F,Campinas F,1.01,13.00,138.5,1.83,1.87,0.0,0.00,0.00,0EqjVVa3,0.990099,0.076923,0.546448,0.534759,0.067022,485.718,847.297026,1.744422,3.0,0.000000,0.000000,71.69,5.048,8.371847,1.658448,130.0,136.730,59.512903,0.435259,1.8,1.643168,0.912871,230.48,2.000,0.979770,0.489885,12.0,67,67,1.07,3.44,83.294,135.136,1.210308,0.015289,NaN,0.21,0.042,0.238095,0.000000,0.0,0.000000,-0.97,-0.194,-61.855670,0.000000,0.0,0.000000
2342,20

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
2350,19:30,Venezuela Superliga,Spartans,Frontinos,1.98,Back Home
2367,05:00,Austrália Nbl1 Norte,Ipswich Force F,Mackay Meteorettes F,2.37,Back Home
2369,07:00,Austrália Nbl1 Sul,Casey Cavaliers,Nunawading,2.62,Back Home
2374,05:00,Austrália Nbl1 Sul,Dandenong Rangers F,Frankston F,2.19,Back Home
2379,09:30,Austrália Nbl1 Oeste,East Perth Eagles,Joondalup Wolves,1.95,Back Home
